In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

2023-03-22 16:25:59.627435: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
cls2idx = {'Others': 0, 'VotePaper': 1, 'VoteBoard': 2, 'ElectoralUnit': 3}

In [3]:
def create_image_classification_dataset(image_paths, labels, image_size, batch_size):    
    # Create a TensorFlow dataset from the image paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # Define a function to load and preprocess each image
    def load_and_preprocess_image(image_path, label):
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, image_size)
        return image, tf.one_hot(label, len(cls2idx))
    
    # Use the map function to apply the load_and_preprocess_image function to each image in the dataset
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    dataset = dataset.batch(batch_size)

    return dataset

In [4]:
def read_csv(csv_file, image_dir,):
    # Read the CSV file into a Pandas dataframe
    df = pd.read_csv(csv_file)
    
    # Create a list of image file paths by joining the directory and filename columns in the dataframe
    image_paths = [os.path.join(image_dir, filename.split('d=')[-1]) for filename in df['image']]
    
    # Create a list of labels by converting the label column in the dataframe to a list of strings
    labels = df['choice'].fillna('Others').astype(str).apply(lambda x: cls2idx[x]).tolist()
    return train_test_split(image_paths, labels)

In [5]:
IMG_SIZE = (224, 224)
x_train, x_test, y_train, y_test = read_csv('label-3-class.csv', 'data',)
train_dataset = create_image_classification_dataset(x_train, y_train, IMG_SIZE, 16)
test_dataset = create_image_classification_dataset(x_test, y_test, IMG_SIZE, 16)

2023-03-22 16:26:10.002558: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV3Large(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input

In [7]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(16, 7, 7, 960)


In [8]:
# base_model.trainable = False

In [9]:
def build_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = inputs
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(len(cls2idx), activation='softmax')(x)
    return tf.keras.Model(inputs, outputs)

In [10]:
model = build_model()

In [11]:
base_learning_rate = 0.0001
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.Recall(), 
            'accuracy']
)

In [12]:
model.fit(train_dataset, validation_data=test_dataset, epochs=5)

Epoch 1/5
26/26 [==============================] - 46s 1s/step - loss: 0.8968 - recall: 0.5481 - accuracy: 0.6543 - val_loss: 0.4857 - val_recall: 0.7778 - val_accuracy: 0.8370
Epoch 2/5
26/26 [==============================] - 37s 1s/step - loss: 0.3700 - recall: 0.8593 - accuracy: 0.8914 - val_loss: 0.3524 - val_recall: 0.8593 - val_accuracy: 0.8815
Epoch 3/5
26/26 [==============================] - 43s 2s/step - loss: 0.1846 - recall: 0.9383 - accuracy: 0.9531 - val_loss: 0.3698 - val_recall: 0.8370 - val_accuracy: 0.8519
Epoch 4/5
26/26 [==============================] - 38s 1s/step - loss: 0.1680 - recall: 0.9457 - accuracy: 0.9506 - val_loss: 0.3781 - val_recall: 0.8519 - val_accuracy: 0.8593
Epoch 5/5
26/26 [==============================] - 37s 1s/step - loss: 0.0891 - recall: 0.9728 - accuracy: 0.9778 - val_loss: 0.2813 - val_recall: 0.9259 - val_accuracy: 0.9259


In [13]:
model.evaluate(test_dataset,)

9/9 [==============================] - 3s 311ms/step - loss: 0.2813 - recall: 0.9259 - accuracy: 0.9259


[0.2812821865081787, 0.9259259104728699, 0.9259259104728699]

In [14]:
y_pred = model.predict(test_dataset,)
y_true = []
for x, y in test_dataset: y_true.extend(y.numpy().argmax(1))

9/9 [==============================] - 4s 319ms/step


In [15]:
y_pred.argmax(1)

array([1, 3, 1, 1, 3, 0, 0, 3, 2, 1, 2, 3, 0, 2, 2, 0, 0, 1, 2, 3, 3, 0,
       0, 3, 2, 3, 2, 1, 0, 2, 2, 2, 0, 0, 3, 3, 2, 3, 2, 2, 3, 3, 3, 3,
       3, 0, 1, 2, 0, 1, 3, 2, 3, 1, 1, 3, 2, 2, 2, 2, 1, 1, 0, 2, 1, 0,
       2, 2, 3, 2, 0, 1, 2, 3, 0, 2, 2, 1, 2, 3, 0, 1, 2, 2, 0, 1, 0, 2,
       1, 2, 3, 2, 0, 1, 2, 2, 2, 0, 0, 1, 3, 0, 2, 2, 2, 2, 3, 2, 3, 2,
       3, 3, 0, 3, 2, 0, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 1, 3,
       2, 0, 2])

In [16]:
print(classification_report(y_true, y_pred.argmax(1)))

              precision    recall  f1-score   support

           0       0.96      0.76      0.85        33
           1       0.92      0.92      0.92        24
           2       0.94      1.00      0.97        51
           3       0.87      1.00      0.93        27

    accuracy                           0.93       135
   macro avg       0.92      0.92      0.92       135
weighted avg       0.93      0.93      0.92       135



In [17]:
confusion_matrix(y_true, y_pred.argmax(1))

array([[25,  2,  2,  4],
       [ 1, 22,  1,  0],
       [ 0,  0, 51,  0],
       [ 0,  0,  0, 27]])

In [18]:
model.save('saved_models/mobile-net')

INFO:tensorflow:Assets written to: saved_models/mobile-net/assets


INFO:tensorflow:Assets written to: saved_models/mobile-net/assets
